<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceSemanticKernelLogo.png" width="60px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Decisions with Multi-AI Agent Personas</span>
</div>

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.5"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.5"

//using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var azureOpenAIReasoningEndpoint = config["AzureOpenAI:ReasoningEndpoint"];
var azureOpenAIReasoningAPIKey = config["AzureOpenAI:ReasoningAPIKey"];
var azureOpenAIReasoningModelDeploymentName = config["AzureOpenAI:ReasoningModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.5 Microsoft.Extensions.Configuration.Json, 9.0.5

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.55" 
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.55"
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, 1.55"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Create a Semantic Kernel with both GeneralAI and ReasoningAI services
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName!,
        endpoint: azureOpenAIEndpoint!,
        apiKey: azureOpenAIAPIKey!,
        serviceId: "GeneralAI")
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIReasoningModelDeploymentName!,
        endpoint: azureOpenAIReasoningEndpoint!,
        apiKey: azureOpenAIReasoningAPIKey!,
        serviceId: "ReasoningAI")
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.55.0 Microsoft.SemanticKernel.Agents.Abstractions, 1.55.0 Microsoft.SemanticKernel.Agents.Core, 1.55.0

### Step 2 - Decisions with Multi-Agent Personas 

> "The many are smarter than the few when it comes to solving problems, predicting outcomes, and innovating solutions, as long as there is diversity of opinion, independence, and decentralization."  
>
> -- <cite>James Surowiecki (Author "The Wisdom of Crowds")</cite> 

Single agent personas are effective constructs in Semantic Kernel, but can largely be replicated in other mechanisms with prompts. Basically, a single AI Agent is not different than creating a system prompt with instructions and even some plugins for function calling. **Creating multiple personas (experts) that can interact with each other is the true power of the Semantic Kernel Agent functionality. This allows multiple agents to be orchestrated in various ways.** 

Execute the cell below to illustrate how two different personas can interact in a decision-making process moderated by a decision intelligence expert that makes the final reasoning conclusiion on the decision task at hand. In the output notice key steps:
* Each expert (Dwight and Ben) make their recommendation, based on their decision persona & decision tools 
* The final recommendation is made based on the synthesis of the agent recommendations by the Decision Maker agent  
* This multi-agent interaction to converge on a single decision mimic the self-consistency GenAI pattern 


> 📝 Note: The execution below can take the some time. Use GPT-4.1-mini to optimize speed of the execution for illustrative purposes. Conversely, long-running agentic executions can also be optimized for accuracy as these ideally would be scheduled as "jobs" or tasks. Therefore, you can use reasoning models in these situations as well.  

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.ChatCompletion;
using System.Threading;

// General AI: Create Prompt Execution Settings & Kernel Arguement
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ServiceId = "GeneralAI", // Use the General AI service for chat
    MaxTokens = 4000, 
    Temperature = 0.4, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

#pragma warning disable SKEXP0110
// Create a new chat TerminationStrategy that will terminate when the final message contains the term "final decision"
private sealed class ApprovalTerminationStrategy : TerminationStrategy
{
    // Terminate when the final message contains the term "approve"
    protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
        => Task.FromResult(history[history.Count - 1].Content?.Contains("final decision", StringComparison.OrdinalIgnoreCase) ?? false);
}

var assistantDwightEisenhower = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    Arguments = kernelArguments,
    Name = "DwightEisenhower", // Dwight Eisenhower persona
    Description = "A decision intelligence agent that uses the Eisenhower Decision Priorotization Matrix.",
    Instructions = "You are a decision intelligence strategist with the knowledge of Dwight Eisenhower's Decision Prioritization Matrix. Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. Provide structured, logical, and comprehensive advice."
};

var assistantBenFranklin = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    Arguments = kernelArguments,
    Name = "BenFranklin", // Ben Franklin persona
    Description = "A decision intelligence agent that uses the Ben Franklin Decision Framework.",
    Instructions = "You are a decision intelligence strategist with the knowledge of Ben Franklin's Decision Framework. Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. Provide structured, logical, and comprehensive advice."
};

var assistantDecisionMaker = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    Arguments = kernelArguments,
    Name = "DecisionMaker",
    Description = "A decision intelligence agent that approves the final decision.",
    Instructions = """
    You are a Decision Intelligence maker. Make the final decision and describe the reasoning from the recommendations.
    You're laser focused on the goal at hand. Don't waste time with chit chat.
    Consider suggestions when refining an idea. 
    When ready to approve the final the decision, use the words final decision to approve the final decision.
    """
};

// Create a chat for agent interaction.
AgentGroupChat chat =
    new(assistantDwightEisenhower, assistantBenFranklin, assistantDecisionMaker)
    {
        ExecutionSettings =
            new()
            {
                // Here a TerminationStrategy subclass is used that will terminate when
                // an assistant message contains the term "approve".
                TerminationStrategy =
                    new ApprovalTerminationStrategy()
                    {
                        // Only the art-director may approve.
                        Agents = [assistantDecisionMaker],
                        // Limit total number of turns
                        MaximumIterations = 10,
                    },
            }
    };

// Invoke chat and display messages.
ChatMessageContent decisionRecommendation = new(AuthorRole.User, """
### Decision To Process:  
I am thinking of buying a rental property versus investing in Bitcoing. 
What things should I consider in making this decision?
### Decision Constraints:  
1) You want to optimize for long-term financial growth, but also mitigate risk. 
2) You have a budget of $500,000 cash for the investment. 
3) You are the in early 40s and the breadwinner of your family and need to ensure financial stability.
4) You don't have much debt, but have kids going to college in the next 5 years.
5) You are not an expert in real estate or cryptocurrency, but have a basic understanding of both.

When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  
""");
chat.AddChatMessage(decisionRecommendation);
decisionRecommendation.Content.DisplayAs("text/markdown");
Console.WriteLine("----------------------------------------------------------");
Console.WriteLine("----------------------------------------------------------");

await foreach (ChatMessageContent response in chat.InvokeAsync())
{
    response.Content.DisplayAs("text/markdown");
}

### Decision To Process:  
I am thinking of buying a rental property versus investing in Bitcoing. 
What things should I consider in making this decision?
### Decision Constraints:  
1) You want to optimize for long-term financial growth, but also mitigate risk. 
2) You have a budget of $500,000 cash for the investment. 
3) You are the in early 40s and the breadwinner of your family and need to ensure financial stability.
4) You don't have much debt, but have kids going to college in the next 5 years.
5) You are not an expert in real estate or cryptocurrency, but have a basic understanding of both.

When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  

### Framing the Decision: Rental Property vs. Bitcoin Investment

You are considering two distinct investment options: purchasing a rental property or investing in Bitcoin. Both have different risk profiles, growth potential, and management requirements. Let’s apply structured decision analysis and the Eisenhower Matrix principles (urgent/important) to clarify your priorities and options.

---

### Key Factors to Consider

#### 1. **Risk Profile & Volatility**
- **Rental Property:** 
  - Generally lower volatility, but subject to real estate market cycles, tenant risk, and property-specific issues.
  - Tangible asset; insurance can mitigate some risks (fire, damage).
- **Bitcoin:** 
  - Highly volatile; prices can swing dramatically in short periods.
  - Regulatory risk and potential for security breaches/theft.
  - No intrinsic value or cash flow; value driven by market sentiment.

#### 2. **Potential Returns & Growth**
- **Rental Property:** 
  - Returns from rental income (cash flow) + property appreciation.
  - Historically, real estate appreciates over time, but growth is generally moderate.
- **Bitcoin:** 
  - Potential for high returns, but also significant downside.
  - No income generation—returns are solely from price appreciation.

#### 3. **Liquidity**
- **Rental Property:** 
  - Illiquid; selling can take months and involves transaction costs.
  - Not easy to access cash quickly if needed.
- **Bitcoin:** 
  - Highly liquid; can sell portions at any time, 24/7 trading.

#### 4. **Time & Management Commitment**
- **Rental Property:** 
  - Requires ongoing management (maintenance, tenants, taxes).
  - Can outsource to property managers (reduces profit).
- **Bitcoin:** 
  - Minimal time commitment after purchase.
  - Requires staying informed about security and regulatory changes.

#### 5. **Tax Implications**
- **Rental Property:** 
  - Rental income is taxable, but you can deduct mortgage interest, depreciation, repairs.
  - Capital gains tax on sale, but possible to defer via 1031 exchange.
- **Bitcoin:** 
  - Taxed as property in the US; capital gains apply on sale.
  - No depreciation or income tax benefits.

#### 6. **Diversification & Portfolio Fit**
- **Rental Property:** 
  - Real estate can diversify a portfolio heavy in stocks/bonds.
- **Bitcoin:** 
  - Adds diversification, but with higher risk; correlation to traditional assets varies.

#### 7. **Family & Financial Stability**
- **Rental Property:** 
  - More predictable cash flow; can help with college expenses.
  - Asset can be passed on or sold for lump sum.
- **Bitcoin:** 
  - Unpredictable; value could drop significantly when you need funds.

---

### Eisenhower Matrix Application

Let’s map your decision elements:

| Urgent             | Not Urgent         |
|--------------------|--------------------|
| **Important**      | - College funding in 5 years<br>- Financial stability for family<br>- Risk mitigation<br>- Long-term growth | - Diversification<br>- Passive income<br>- Tax optimization |
| **Not Important**  | - Following trends<br>- Speculative returns | - Complex management (if avoidable)<br>- High time commitment |

**Your focus should be on important and urgent factors:** financial stability, risk mitigation, and college funding.

---

### Decision Analysis Table

| Factor                | Rental Property         | Bitcoin                |
|-----------------------|------------------------|------------------------|
| **Risk/Volatility**   | Lower                  | High                   |
| **Potential Returns** | Moderate, steady       | High, unpredictable    |
| **Liquidity**         | Low                    | High                   |
| **Management**        | Moderate/High          | Low                    |
| **Tax Benefits**      | Yes                    | Minimal                |
| **Diversification**   | Yes                    | Yes, but risky         |
| **Stability**         | High                   | Low                    |
| **College Funding**   | Reliable               | Unreliable             |

---

### Systems Thinking: Interdependencies and Scenarios

- **Rental Property:** Provides steady income, can be leveraged for loans, and offers inflation protection. However, requires time or management fees and exposes you to local market risks.
- **Bitcoin:** Offers potential for rapid appreciation, but value could be much lower when you need it for college expenses. Minimal management, but high emotional and financial volatility.

---

### Recommendations & Next Steps

#### 1. **Clarify Your Priorities**
- If your top priority is financial stability and college funding, lean toward rental property.
- If you are willing to accept high risk for potential high reward, consider a (smaller) allocation to Bitcoin.

#### 2. **Consider Diversification**
- You don’t have to choose only one. For example, invest $400,000 in real estate and $100,000 in Bitcoin to balance stability and growth potential.

#### 3. **Risk Mitigation**
- For rental property, consider hiring a property manager and purchasing in a stable market.
- For Bitcoin, use secure wallets and only invest what you can afford to lose.

#### 4. **Scenario Planning**
- Run scenarios: What happens if the property is vacant for 6 months? What if Bitcoin drops 50% before you need the money?

#### 5. **Professional Advice**
- Consult a financial advisor and tax professional to optimize your investment and tax strategy.

---

### Summary Table

| Priority              | Best Option         | Why?                    |
|-----------------------|--------------------|-------------------------|
| Financial Stability   | Rental Property    | Predictable cash flow, lower risk |
| High Growth Potential | Bitcoin            | Potential for high returns, but risky |
| Liquidity             | Bitcoin            | Easily sold if needed   |
| College Funding       | Rental Property    | More reliable timeline  |
| Diversification       | Both (split funds) | Balances risk and reward|

---

### Final Thought

Given your constraints and priorities, a **weighted approach** (e.g., 80% rental property, 20% Bitcoin) may provide both stability and growth potential, while keeping risk manageable. Regularly review your portfolio as your needs and market conditions change.

### Decision Factors to Consider

#### Financial Growth Potential
- **Rental Property:** Offers rental income and potential for property appreciation. Historically steady, but growth rates depend on location and market cycles.
- **Bitcoin:** High potential returns, but with significant volatility. Past performance has been strong, but future growth is uncertain and speculative.

#### Risk Profile
- **Rental Property:** Subject to property market fluctuations, tenant risk, maintenance costs, and potential vacancies. Generally less volatile than cryptocurrency.
- **Bitcoin:** Extremely volatile; prices can swing dramatically in short periods. Regulatory risk and potential for loss due to hacking or mismanagement.

#### Liquidity
- **Rental Property:** Illiquid; selling can take months and involves significant transaction costs.
- **Bitcoin:** Highly liquid; can be sold quickly and in increments.

#### Income Generation
- **Rental Property:** Provides ongoing rental income, which can help with cash flow and college expenses.
- **Bitcoin:** No income; only capital gains (if any) upon sale.

#### Time & Expertise Required
- **Rental Property:** Requires property management, dealing with tenants, maintenance, and legal compliance. Can hire a property manager for a fee.
- **Bitcoin:** Requires ongoing monitoring of the market and security of digital assets, but less day-to-day involvement.

#### Tax Implications
- **Rental Property:** Rental income is taxable, but you can deduct expenses (mortgage interest, depreciation, repairs). Capital gains tax on sale, with possible 1031 exchange benefits.
- **Bitcoin:** Taxed as property; capital gains tax applies when sold. No ongoing income or deductions.

#### Diversification & Portfolio Impact
- **Rental Property:** Adds real estate exposure to your portfolio, which can diversify away from stocks/bonds.
- **Bitcoin:** Adds a non-traditional asset; correlation with other assets varies, but increases overall portfolio risk.

#### Family & Stability Considerations
- **Rental Property:** More predictable income and value; can be leveraged for loans or sold for college expenses.
- **Bitcoin:** Value could be much lower when you need funds; not a reliable source for planned expenses.

---

### Systems Thinking: Interconnections & Scenarios

- **Rental property** can provide steady cash flow, potential appreciation, and tax benefits, but involves management and market risks.
- **Bitcoin** offers high growth potential but with high risk and no income, and its value may not align with your college funding timeline.
- Your age, family status, and upcoming college expenses suggest a need for stability and predictable cash flow.
- Both options could be combined for diversification (e.g., 80% property, 20% Bitcoin).

---

### Decision Table

| Factor                     | Rental Property        | Bitcoin               |
|----------------------------|-----------------------|-----------------------|
| Growth Potential           | Moderate, steady      | High, speculative     |
| Risk                       | Moderate              | High                  |
| Liquidity                  | Low                   | High                  |
| Income                     | Yes                   | No                    |
| Management                 | High                  | Low                   |
| Tax Benefits               | Yes                   | Minimal               |
| Diversification            | Yes                   | Yes                   |
| Stability for Family       | High                  | Low                   |
| College Funding Alignment  | Good                  | Poor                  |

---

### Recommendations

1. **Clarify Priorities:**  
   - If stability, predictable cash flow, and college funding are top priorities, rental property is more suitable.
   - If you are comfortable with high risk and want to pursue potential high returns, Bitcoin could be a smaller allocation.

2. **Consider Diversification:**  
   - You don’t have to choose only one. Consider a split (e.g., $400k property, $100k Bitcoin) to balance risk and reward.

3. **Risk Management:**  
   - For property: Choose a good location, consider professional management, and ensure adequate insurance.
   - For Bitcoin: Use secure wallets, only invest what you can afford to lose, and stay updated on regulations.

4. **Scenario Planning:**  
   - What if property values drop or you have vacancies?
   - What if Bitcoin crashes before college expenses are due?
   - How quickly can you access funds if needed?

5. **Professional Advice:**  
   - Consult a financial advisor and tax professional to tailor the investment to your specific situation.

---

### Summary

Given your goals and constraints, **rental property** aligns best with your need for stability, income, and college funding. A **small allocation to Bitcoin** could provide growth potential without jeopardizing your family’s financial security. Regularly review and adjust your strategy as your needs and market conditions evolve.

### Decision Analysis

#### Key Considerations
- **Risk Mitigation:** Rental property offers lower volatility and more predictable returns; Bitcoin is highly volatile and speculative.
- **Financial Growth:** Bitcoin has higher potential returns but with greater risk; rental property offers moderate, steady growth plus income.
- **Liquidity:** Bitcoin is highly liquid; property is illiquid.
- **Income:** Rental property generates ongoing income; Bitcoin does not.
- **Time Commitment:** Rental property requires management; Bitcoin is passive.
- **Tax Implications:** Rental property offers deductions and depreciation; Bitcoin offers no ongoing tax benefits.
- **Timeline:** College expenses in 5 years require reliability; property income is more predictable than Bitcoin price appreciation.
- **Expertise:** You have only basic knowledge of both; property management can be outsourced, but both require some learning.

#### Recommendations Considered
- Rental property aligns with your need for stability, income, and predictable cash flow for upcoming college expenses.
- Bitcoin could be a small, speculative allocation for growth, but not the core of your strategy.
- Diversification (e.g., 80% property, 20% Bitcoin) balances stability and growth potential.

### Final Decision

Invest the majority of your funds ($400,000–$450,000) in a rental property in a stable market, using professional property management to reduce your time commitment and risk. Allocate a small portion ($50,000–$100,000) to Bitcoin for growth potential, but treat it as a speculative, long-term holding. This approach optimizes for long-term financial growth, mitigates risk, supports family stability, and provides reliable income for future college expenses.

**Final decision:** Prioritize a rental property investment for stability and income, with a minor allocation to Bitcoin for diversification and growth potential.

### Step 3 - Advanced Multi-Agent Decision Orchestration  

In [ ]:
// Import the new Magentic orchestration package and Semantic Kernel InProcessRuntime
#r "nuget: Microsoft.SemanticKernel.Agents.Magentic, 1.55.0-preview" 
#r "nuget: Microsoft.SemanticKernel.Agents.Orchestration, 1.55.0-preview" 
#r "nuget: Microsoft.SemanticKernel.Agents.Runtime.InProcess, 1.55.0-preview" 

using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Magentic;
using Microsoft.SemanticKernel.Agents.Orchestration;
using Microsoft.SemanticKernel.Agents.Runtime.InProcess;

// Create a Devil's advocate agent that rigorously tests and challenges recommendations
var assistantDevilsAdvocate = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    Name = "DevilsAdvocate",
    Description = "A decision intelligence agent that acts as a devil's advocate, rigorously testing and challenging recommendations.",
    Instructions = @"
You are a devil’s advocate decision intelligence strategist. 
Your job is to critically evaluate any proposed recommendations or decisions by:
  1. Identifying hidden assumptions and blind spots  
  2. Raising potential risks, unintended consequences, and edge cases  
  3. Questioning feasibility, resource constraints, and stakeholder buy-in  
  4. Suggesting alternative viewpoints or counter-arguments  
  5. Prompting further data, evidence, or analysis where needed  
Respond in a structured way: state each challenge clearly, explain why it matters, 
and recommend how to address or investigate it further."
};

#pragma warning disable SKEXP0010
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
        //MaxTokens = 95000,
        ReasoningEffort = "High", // Set to High for reasoning
        ServiceId = "ReasoningAI"
    };

#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001
var magenticManager = new StandardMagenticManager(
    // Use Reasoning AI for manager orchestration
    semanticKernel.GetRequiredService<IChatCompletionService>(),
    openAIPromptExecutionSettings)
{
    MaximumInvocationCount = 10, // Maximum number of invocations per orchestration
};


// Initalize the ChatHistory to store the messages exchanged during the orchestration
ChatHistory history = [];

// Define a response callback to handle the responses from the orchestration
// All the agent messages will be added in the history
ValueTask responseCallback(ChatMessageContent response)
{
    history.Add(response);
    return ValueTask.CompletedTask;
}

// Create the MagenticOrchestration with the agents and the response callback
MagenticOrchestration orchestration = new MagenticOrchestration(
    magenticManager,
    assistantDevilsAdvocate,
    assistantDwightEisenhower,
    assistantBenFranklin)
{
    ResponseCallback = responseCallback,
};

// Create an Semantic Kernel InProcessRuntime to run the orchestration
InProcessRuntime runtime = new InProcessRuntime();
await runtime.StartAsync();

string decisionToProcess = """
### Decision To Process:  
I am thinking of buying a rental property versus investing in Bitcoing. 
What things should I consider in making this decision?
### Decision Constraints:  
1) You want to optimize for long-term financial growth, but also mitigate risk. 
2) You have a budget of $500,000 cash for the investment. 
3) You are the in early 40s and the breadwinner of your family and need to ensure financial stability.
4) You don't have much debt, but have kids going to college in the next 5 years.
5) You are not an expert in real estate or cryptocurrency, but have a basic understanding of both.

When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  
""";
decisionToProcess.DisplayAs("text/markdown");
Console.WriteLine("----------------------------------------------------------");
Console.WriteLine("----------------------------------------------------------");
"### FINAL RECOMMENDATION".DisplayAs("text/markdown");

// Invoke the orchestration with the decision to process
var orchestrationResult = await orchestration.InvokeAsync(decisionToProcess, runtime);
// Display the decision recommendation from the orchestration result with a timeout of 300 seconds
string decisionRecommendation = await orchestrationResult.GetValueAsync(TimeSpan.FromSeconds(300));
decisionRecommendation.DisplayAs("text/markdown");

Console.WriteLine("----------------------------------------------------------");
Console.WriteLine("----------------------------------------------------------");
"### ORCHESTRATION HISTORY".DisplayAs("text/markdown");

// Show the history of messages exchanged during the orchestration
foreach (ChatMessageContent message in history)
{
    var messageString = $"""

    
    ### {message.Role} - {message.AuthorName}: 
    {message.Content}
    """;
    messageString.DisplayAs("text/markdown");
}

// Wait for the runtime to complete all tasks
// Dispose the runtime to clean up resources
await runtime.RunUntilIdleAsync();
await runtime.DisposeAsync()

Installed Packages Microsoft.SemanticKernel.Agents.Magentic, 1.55.0-preview Microsoft.SemanticKernel.Agents.Orchestration, 1.55.0-preview Microsoft.SemanticKernel.Agents.Runtime.InProcess, 1.55.0-preview

### Decision To Process:  
I am thinking of buying a rental property versus investing in Bitcoing. 
What things should I consider in making this decision?
### Decision Constraints:  
1) You want to optimize for long-term financial growth, but also mitigate risk. 
2) You have a budget of $500,000 cash for the investment. 
3) You are the in early 40s and the breadwinner of your family and need to ensure financial stability.
4) You don't have much debt, but have kids going to college in the next 5 years.
5) You are not an expert in real estate or cryptocurrency, but have a basic understanding of both.

When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  

----------------------------------------------------------
----------------------------------------------------------


### FINAL RECOMMENDATION

### Synthesis: Weighted Scores vs. Risk Challenges

#### What the Weighted Scores Show  
- **Rental Property**: Strongly favored for stability, moderate returns, and cash flow, especially given your family and upcoming college expenses.  
- **Bitcoin**: High potential returns and liquidity, but extreme volatility and risk, with significant downside in worst‐case scenarios.  

#### What the Risk Challenges Reveal  
- **Rental Property Risks**: Illiquidity, management burden, and potential for unexpected costs or market downturns. Cash flow is not guaranteed, and property may not sell quickly if funds are urgently needed.  
- **Bitcoin Risks**: Value can drop sharply when you need money most, regulatory changes could impact access or taxation, and behavioral biases may lead to poor timing decisions.  

#### Trade-offs  
- **Stability vs. Growth**: Rental property offers more predictable outcomes but less explosive upside. Bitcoin could outperform but is much riskier and not ideal for near-term obligations.  
- **Liquidity Needs**: College expenses in 5 years require a portion of your portfolio to be reliably accessible and not subject to large drawdowns.  
- **Management Time**: Rental property requires active management or outsourcing; Bitcoin requires security diligence and tax tracking.  

---

### Recommendation Framework

1. **Protect College Funding**  
   Set aside a dedicated, liquid fund (e.g., high-yield savings, short-term bonds/CDs) for at least the first year of college expenses. This ensures you are not forced to liquidate risk assets at a loss.

2. **Core Allocation: Rental Property**  
   Allocate the majority (e.g., 60–80%) of your investable funds to a rental property, emphasizing stable cash flow, conservative vacancy/maintenance assumptions, and professional management. Consider market research to select a resilient location and property type.

3. **Satellite Allocation: Bitcoin**  
   Allocate a smaller portion (e.g., 10–20%) to Bitcoin, recognizing its role as a speculative, high-growth asset. Use secure custody solutions and set rules for periodic rebalancing or profit-taking. Only invest what you are prepared to lose without jeopardizing family stability.

4. **Diversification and Flexibility**  
   Consider supplementing with REITs or other liquid, income-producing assets for additional diversification and liquidity. Revisit your allocation annually or after major life changes.

5. **Professional Guidance**  
   Consult a financial planner and tax advisor to refine allocations, stress-test scenarios, and optimize for after-tax returns.

---

### Final Recommendation

**Given your goals, constraints, and risk profile, a blended approach is optimal:**  
- **Prioritize stability and liquidity for near-term obligations (college).**  
- **Use rental property as your core long-term growth and income driver.**  
- **Limit Bitcoin exposure to a level that won’t threaten your family’s financial security if it underperforms.**

**Next Steps:**  
1. Quantify your minimum liquidity needs for college.  
2. Research local rental markets and interview property managers.  
3. Decide on a prudent Bitcoin allocation and secure storage.  
4. Schedule a session with a fiduciary advisor for scenario planning.

...

### ORCHESTRATION HISTORY

### Assistant - DwightEisenhower: 
### Restating Your Objectives and Constraints

- **Primary Objective:** Optimize for long-term financial growth while mitigating risk.
- **Budget:** $500,000 cash available for investment.
- **Personal Situation:** Early 40s, breadwinner, aiming for financial stability.
- **Upcoming Expenses:** Children will be attending college in the next 5 years.
- **Experience:** Basic understanding of both real estate and cryptocurrency; not an expert in either.
- **Debt:** Minimal.

---

### Key Decision Criteria to Consider

#### 1. Expected Return on Investment (ROI)
- **Rental Property:** Historically stable, moderate returns (rental income + property appreciation).
- **Bitcoin:** High potential returns, but with significant volatility and unpredictability.

#### 2. Risk and Volatility
- **Rental Property:** Market risk (local real estate trends), tenant risk, property-specific risks.
- **Bitcoin:** Extreme price volatility, regulatory risk, potential for significant loss.

#### 3. Cash Flow and Liquidity
- **Rental Property:** Provides regular rental income, but property is illiquid (selling takes time).
- **Bitcoin:** Highly liquid (can sell quickly), but no regular cash flow.

#### 4. Time Horizon and Upcoming Expenses
- **Rental Property:** Can generate income to help with college costs, but may not be easily liquidated on short notice.
- **Bitcoin:** Easily sold if needed, but value could be much lower when you need funds.

#### 5. Management and Expertise Required
- **Rental Property:** Requires property management, dealing with tenants, maintenance, legal compliance.
- **Bitcoin:** Requires understanding of digital wallets, security, market trends, and tax reporting.

#### 6. Diversification and Portfolio Impact
- **Rental Property:** Adds real asset exposure, diversifies away from stocks/bonds.
- **Bitcoin:** Adds alternative asset exposure, but may increase overall portfolio volatility.

#### 7. Tax Implications
- **Rental Property:** Rental income is taxable; potential deductions for expenses; capital gains tax on sale.
- **Bitcoin:** Taxed as property; capital gains tax applies on sale; complex reporting requirements.

#### 8. Leverage Opportunities
- **Rental Property:** Can use mortgage to amplify returns (though you’re considering cash purchase).
- **Bitcoin:** Typically not leveraged by most retail investors due to risk.

---

### Eisenhower Matrix Application

#### Urgent and Important
- Ensuring liquidity for college expenses (next 5 years).
- Maintaining financial stability as the primary earner.

#### Important, Not Urgent
- Long-term growth and wealth accumulation.
- Diversification of investment portfolio.

#### Not Important, Urgent
- Short-term speculation or chasing recent trends.

#### Not Important, Not Urgent
- Investments outside your expertise that require significant time to learn/manage.

---

### Summary Table

| Criteria                | Rental Property                | Bitcoin                       |
|-------------------------|-------------------------------|-------------------------------|
| Expected Return         | Moderate, stable               | High, highly volatile         |
| Risk                    | Moderate, property/market risk | Very high, market/regulatory  |
| Cash Flow               | Yes, rental income             | No, unless trading            |
| Liquidity               | Low                            | High                          |
| Management Burden       | High (unless outsourced)       | Low, but requires vigilance   |
| Tax Complexity          | Moderate                       | High                          |
| Diversification         | Real asset                     | Digital/alternative asset     |
| Suitability for College | Moderate (if income reliable)  | Low (value may fluctuate)     |

---

### Next Steps

1. **Clarify Your Priorities:**  
   - How much liquidity do you need for college expenses?  
   - How much risk can you tolerate?

2. **Consider a Blended Approach:**  
   - Is a mix (e.g., 80% property, 20% Bitcoin) possible to balance growth and risk?

3. **Assess Your Willingness to Learn/Manage:**  
   - Are you prepared to manage a property or learn more about crypto security/taxation?

4. **Consult Professionals:**  
   - Speak with a financial advisor, tax professional, and possibly a property manager.

---

### Decision Intelligence Takeaway

Given your constraints and objectives, rental property may better align with your need for stability, cash flow, and moderate risk—especially with upcoming college expenses. Bitcoin could play a smaller, speculative role in your portfolio if you’re comfortable with its risks and volatility.

Would you like a decision tree or scenario analysis to further clarify the trade-offs?

### Assistant - BenFranklin: 
### Criteria-Based Pros and Cons

#### Rental Property

| Criteria                | Pros                                                                 | Cons                                                               | Weight (1-5) | Score (Pros +, Cons -) |
|-------------------------|---------------------------------------------------------------------|--------------------------------------------------------------------|--------------|-----------------------|
| Expected Return         | Historically stable appreciation + rental income                     | Market cycles can affect value/appreciation                        | 5            | +4, -2                |
| Risk                    | Tangible asset, less volatile than crypto                           | Local market risk, tenant risk, property damage                    | 5            | +4, -2                |
| Cash Flow               | Regular rental income can help with college expenses                 | Vacancies, maintenance costs can interrupt cash flow               | 4            | +4, -2                |
| Liquidity               | Potential for refinancing, can sell if needed (but slowly)           | Illiquid, may take months to sell                                  | 4            | +2, -4                |
| Management Burden       | Can outsource property management                                   | Time-consuming, landlord responsibilities                          | 3            | +2, -3                |
| Tax Complexity          | Depreciation, deductible expenses                                   | Tax reporting, capital gains on sale                               | 3            | +3, -2                |
| Diversification         | Real asset, uncorrelated with stocks/crypto                         | Real estate market can be cyclical                                 | 3            | +3, -1                |
| Suitability for College | Predictable income stream                                           | Not easily liquidated for large, sudden expenses                   | 5            | +4, -3                |

#### Bitcoin

| Criteria                | Pros                                                                 | Cons                                                               | Weight (1-5) | Score (Pros +, Cons -) |
|-------------------------|---------------------------------------------------------------------|--------------------------------------------------------------------|--------------|-----------------------|
| Expected Return         | Potential for very high returns                                     | High volatility, no guarantee of appreciation                      | 5            | +5, -5                |
| Risk                    | Highly liquid, global asset                                         | Extreme price swings, regulatory uncertainty                       | 5            | +2, -5                |
| Cash Flow               | Can sell portions any time                                          | No regular income, only gains if price rises                       | 4            | +3, -4                |
| Liquidity               | Can be sold instantly                                               | Price at time of sale may be unfavorable                           | 4            | +5, -2                |
| Management Burden       | No tenants, low ongoing effort                                      | Security risks (hacks, lost keys), need to track for taxes         | 3            | +3, -2                |
| Tax Complexity          | Only taxed on gains when sold                                       | Complex reporting, IRS scrutiny                                    | 3            | +2, -3                |
| Diversification         | Uncorrelated with traditional assets                                | Highly correlated with crypto market sentiment                     | 3            | +3, -2                |
| Suitability for College | Can liquidate for tuition quickly                                   | Value could drop sharply when funds are needed                     | 5            | +4, -5                |

---

### Weighted Point-Sum Comparison

#### Rental Property

- Expected Return: (4-2) x 5 = **10**
- Risk: (4-2) x 5 = **10**
- Cash Flow: (4-2) x 4 = **8**
- Liquidity: (2-4) x 4 = **-8**
- Management Burden: (2-3) x 3 = **-3**
- Tax Complexity: (3-2) x 3 = **3**
- Diversification: (3-1) x 3 = **6**
- Suitability for College: (4-3) x 5 = **5**

**Total = 10 + 10 + 8 - 8 - 3 + 3 + 6 + 5 = 31**

#### Bitcoin

- Expected Return: (5-5) x 5 = **0**
- Risk: (2-5) x 5 = **-15**
- Cash Flow: (3-4) x 4 = **-4**
- Liquidity: (5-2) x 4 = **12**
- Management Burden: (3-2) x 3 = **3**
- Tax Complexity: (2-3) x 3 = **-3**
- Diversification: (3-2) x 3 = **3**
- Suitability for College: (4-5) x 5 = **-5**

**Total = 0 - 15 - 4 + 12 + 3 - 3 + 3 - 5 = -9**

---

### Summary Table

| Option           | Weighted Score |
|------------------|---------------|
| Rental Property  | 31            |
| Bitcoin          | -9            |

---

### Interpretation

- **Rental Property** scores significantly higher under your criteria, especially considering your need for stability, cash flow, and moderate risk.
- **Bitcoin** offers high liquidity and potential for outsize returns, but its volatility and risk profile make it less suitable as a primary investment for your situation.

**You may wish to consider a blended approach (e.g., 80% property, 20% Bitcoin) if you want some exposure to high-growth assets while maintaining stability.**

### Assistant - DevilsAdvocate: 
### Challenge 1: Oversimplified Scoring Model  
Assumption/Risk  
- The point-sum approach treats each criterion as independent, linear, and perfectly weighted for your personal preferences. In reality:  
  • Criteria interact (e.g., higher leverage boosts returns but also amplifies risk).  
  • Weights and scores are subjective and untested.  

Why It Matters  
- An inaccurate weighting can flip the outcome. You might under- or over-invest in one asset class based on flawed inputs.  

How to Mitigate  
- Conduct a sensitivity analysis: vary each weight ±20% and observe outcome shifts.  
- Consider a full multi-criteria decision analysis (MCDA) tool or decision-tree modeling with probabilistic inputs.  
- Engage a financial planner to validate weights against your true risk tolerance.  

### Challenge 2: Return Assumptions & Downside Scenarios  
Assumption/Risk  
- Rental returns are “moderate, stable” and Bitcoin returns are “high.”  
- Worst-case: real estate crash (e.g., 2008-style downturn) or crypto collapse (e.g., exchange failures, regulatory ban).  

Why It Matters  
- Overestimating upside or underestimating drawdowns leads to insufficient reserves and potential margin calls or liquidity crises.  

How to Mitigate  
- Run scenario analyses using historical peak-to-trough declines:  
  • Real estate: –35% to –45% in hard markets.  
  • Bitcoin: –70%+ in bear cycles.  
- Stress-test your portfolio for simultaneous drops (e.g., recession + crypto bear).  
- Plan stop-loss or cash-reserve buffers (at least 6–12 months of property expenses and college payments).  

### Challenge 3: Correlation & Diversification Blind Spot  
Assumption/Risk  
- Treating real estate and Bitcoin as uncorrelated can be misleading in systemic crises when all asset classes sell off.  

Why It Matters  
- A sudden liquidity crunch could hit both real estate (lack of buyers) and Bitcoin (marketwide sell pressure) simultaneously, impairing your ability to raise funds.  

How to Mitigate  
- Include a truly liquid, low-volatility allocation (e.g., money market funds, short-term Treasuries) earmarked for tuition and emergency cash needs.  
- Quantify tail‐correlation by examining historical data during past market crises.  

### Challenge 4: Cash Flow Reliability & Liquidity Timing  
Assumption/Risk  
- Rental cash flows reliably cover expenses; Bitcoin is instantly liquid at a fair price when needed.  

Why It Matters  
- Prolonged vacancies, unexpected repairs, mortgage rate resets, or exchange downtime/withdrawal limits can disrupt your cash flow and liquidity precisely when college bills arrive.  

How to Mitigate  
- Maintain a dedicated “tuition fund” in a high-yield savings vehicle equal to at least one year of expected costs.  
- Use conservative vacancy and maintenance assumptions (e.g., 15% vacancy, 5–10% gross rent for CapEx).  
- Test crypto withdrawal procedures across multiple platforms to ensure access during high-volatility periods.  

### Challenge 5: Management Burden & Costs Underestimated  
Assumption/Risk  
- Outsourcing property management is straightforward and cost-effective; crypto custody is “set-and-forget.”  

Why It Matters  
- Management fees (8–12% of rent), vacancy fill times, regulatory compliance, insurance, and emergency repairs can erode net returns.  
- Crypto security (cold storage, multi-sig wallets) and tax reporting can consume significant time or require professional help.  

How to Mitigate  
- Get competitive quotes from 2–3 property managers and model their fees into your cash-flow projections.  
- Factor in property insurance, legal fees, and emergency repair reserves as fixed line items.  
- Budget for a crypto security audit or professional custodial service if holdings exceed your comfort level.  

### Challenge 6: Tax & Regulatory Uncertainties  
Assumption/Risk  
- Current tax treatment and regulations will remain stable for both assets.  

Why It Matters  
- Real estate tax benefits (1031 exchanges, depreciation) may face caps or phase-outs.  
- Crypto could face stricter regulations, reporting requirements, or capital controls that affect liquidity and returns.  

How to Mitigate  
- Consult a tax advisor to stress-test after-tax returns under alternative scenarios (e.g., 30% capital gains rate, limitations on 1031 exchanges).  
- Monitor policy proposals affecting crypto (e.g., IRS Form 1099-B reporting, SEC classification).  

### Challenge 7: Stakeholder Buy-in & Behavioral Biases  
Assumption/Risk  
- You alone make decisions, and family will adapt. Overconfidence bias in your ability to manage both investments.  

Why It Matters  
- Family financial stress can escalate if investments underperform. Behavioral biases (panic selling crypto in a crash or over-leveraging property) can magnify losses.  

How to Mitigate  
- Discuss risk profiles and worst-case scenarios with your spouse/partner and possibly older children.  
- Commit to a pre-defined rebalancing or stop-loss plan to curb emotional decision-making.  

### Challenge 8: Alternative & Hybrid Strategies  
Assumption/Risk  
- A pure play in either asset is optimal.  

Why It Matters  
- Excluding other vehicles (REITs, real estate crowdfunding, bond ladders, balanced equity funds, or a blend of Bitcoin and stablecoins) may limit diversification and flexibility.  

How to Mitigate  
- Explore a hybrid approach, such as:  
  • 60% direct real estate, 20% REITs, 20% conservative crypto/stablecoin yield strategies.  
  • Laddered bond/CD portfolio for college expenses.  
- Run Monte Carlo simulations on different allocation mixes to find an efficient risk-adjusted portfolio frontier.  

---  
By systematically challenging these assumptions, you’ll surface hidden risks, improve your decision framework, and arrive at a more resilient, stakeholder-aligned investment plan.

### Assistant - DwightEisenhower: 
### Synthesis: Weighted Scores vs. Risk Challenges

#### What the Weighted Scores Show
- **Rental Property**: Strongly favored for stability, moderate returns, and cash flow, especially given your family and upcoming college expenses.
- **Bitcoin**: High potential returns and liquidity, but extreme volatility and risk, with significant downside in worst-case scenarios.

#### What the Risk Challenges Reveal
- **Rental Property Risks**: Illiquidity, management burden, and potential for unexpected costs or market downturns. Cash flow is not guaranteed, and property may not sell quickly if funds are urgently needed.
- **Bitcoin Risks**: Value can drop sharply when you need money most, regulatory changes could impact access or taxation, and behavioral biases may lead to poor timing decisions.

#### Trade-offs
- **Stability vs. Growth**: Rental property offers more predictable outcomes but less explosive upside. Bitcoin could outperform but is much riskier and not ideal for near-term obligations.
- **Liquidity Needs**: College expenses in 5 years require a portion of your portfolio to be reliably accessible and not subject to large drawdowns.
- **Management Time**: Rental property requires active management or outsourcing; Bitcoin requires security diligence and tax tracking.

---

### Recommendation Framework

#### 1. **Protect College Funding**
   - Set aside a dedicated, liquid fund (e.g., high-yield savings, short-term bonds/CDs) for at least the first year of college expenses. This ensures you are not forced to liquidate risk assets at a loss.

#### 2. **Core Allocation: Rental Property**
   - Allocate the majority (e.g., 60–80%) of your investable funds to a rental property, emphasizing stable cash flow, conservative vacancy/maintenance assumptions, and professional management.
   - Consider market research to select a resilient location and property type.

#### 3. **Satellite Allocation: Bitcoin**
   - Allocate a smaller portion (e.g., 10–20%) to Bitcoin, recognizing its role as a speculative, high-growth asset.
   - Use secure custody solutions and set rules for periodic rebalancing or profit-taking.
   - Only invest what you are prepared to lose without jeopardizing family stability.

#### 4. **Diversification and Flexibility**
   - Consider supplementing with REITs or other liquid, income-producing assets for additional diversification and liquidity.
   - Revisit your allocation annually or after major life changes.

#### 5. **Professional Guidance**
   - Consult a financial planner and tax advisor to refine allocations, stress-test scenarios, and optimize for after-tax returns.

---

### Final Recommendation

**Given your goals, constraints, and risk profile, a blended approach is optimal:**
- **Prioritize stability and liquidity for near-term obligations (college).**
- **Use rental property as your core long-term growth and income driver.**
- **Limit Bitcoin exposure to a level that won’t threaten your family’s financial security if it underperforms.**

**Next Steps:**
1. Quantify your minimum liquidity needs for college.
2. Research local rental markets and interview property managers.
3. Decide on a prudent Bitcoin allocation and secure storage.
4. Schedule a session with a fiduciary advisor for scenario planning.

This approach balances growth, risk, and liquidity—supporting both your family’s stability and your long-term wealth goals.